In [2]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv()

# pip install geopy
from geopy.geocoders import Nominatim
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup
from tqdm import tqdm
# Requests
import requests
import asyncio
import pandas as pd
import numpy as np

from time import sleep
import random
pd.set_option("display.max_columns", None) #Ver todo
import src.SupportLab4 
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


In [ ]:
# Ejecutar el script de manera asíncrona
if __name__ == "__main__":
    df = asyncio.run(main())
    df.to_csv("datos/holabb.csv")
    
    


# Primero de todo usaremos geopy para formatear el nombre de los municipios


In [3]:
lista_municipios = ['acebeda-la', 'ajalvir', 'alameda-del-valle', 'alamo-el', 'alcala-de-henares', 'alcobendas', 'alcorcon', 'aldea-del-fresno', 'algete', 'alpedrete', 'ambite', 'anchuelo', 'aranjuez', 'arganda-del-rey', 'arroyomolinos', 'atazar-el', 'batres', 'becerril-de-la-sierra', 'belmonte-de-tajo', 'berrueco-el', 'berzosa-del-lozoya', 'boadilla-del-monte', 'boalo-el', 'braojos', 'brea-de-tajo', 'brunete', 'buitrago-del-lozoya', 'bustarviejo', 'cabanillas-de-la-sierra', 'cabrera-la', 'cadalso-de-los-vidrios', 'camarma-de-esteruelas', 'campo-real', 'canencia', 'carabana', 'casarrubuelos', 'cenicientos', 'cercedilla', 'cervera-de-buitrago', 'chapineria', 'chinchon', 'ciempozuelos', 'cobena', 'collado-mediano', 'collado-villalba', 'colmenar-del-arroyo', 'colmenar-de-oreja', 'colmenarejo', 'colmenar-viejo', 'corpa', 'coslada', 'cubas-de-la-sagra', 'daganzo-de-arriba', 'escorial-el', 'estremera', 'fresnedillas-de-la-oliva', 'fresno-de-torote', 'fuenlabrada', 'fuente-el-saz-de-jarama', 'fuentiduena-de-tajo', 'galapagar', 'garganta-de-los-montes', 'gargantilla-del-lozoya-y-pinilla-de-buitrago', 'gascones', 'getafe', 'grinon', 'guadalix-de-la-sierra', 'guadarrama', 'hiruela-la', 'horcajo-de-la-sierra-aoslos', 'horcajuelo-de-la-sierra', 'hoyo-de-manzanares', 'humanes-de-madrid', 'leganes', 'loeches', 'lozoya', 'lozoyuela-navas-sieteiglesias', 'madarcos', 'madrid', 'majadahonda', 'manzanares-el-real', 'meco', 'mejorada-del-campo', 'miraflores-de-la-sierra', 'molar-el', 'molinos-los', 'montejo-de-la-sierra', 'moraleja-de-enmedio', 'moralzarzal', 'morata-de-tajuna', 'mostoles', 'navacerrada', 'navalafuente', 'navalagamella', 'navalcarnero', 'navarredonda-y-san-mames', 'navas-del-rey', 'nuevo-baztan', 'olmeda-de-las-fuentes', 'orusco-de-tajuna', 'paracuellos-de-jarama', 'parla', 'patones', 'pedrezuela', 'pelayos-de-la-presa', 'perales-de-tajuna', 'pezuela-de-las-torres', 'pinilla-del-valle', 'pinto', 'pinuecar-gandullas', 'pozuelo-de-alarcon', 'pozuelo-del-rey', 'pradena-del-rincon', 'puebla-de-la-sierra', 'puentes-viejas-manjiron', 'quijorna', 'rascafria', 'reduena', 'ribatejada', 'rivas-vaciamadrid', 'robledillo-de-la-jara', 'robledo-de-chavela', 'robregordo', 'rozas-de-madrid-las', 'rozas-de-puerto-real', 'san-agustin-del-guadalix', 'san-fernando-de-henares', 'san-lorenzo-de-el-escorial', 'san-martin-de-la-vega', 'san-martin-de-valdeiglesias', 'san-sebastian-de-los-reyes', 'santa-maria-de-la-alameda', 'santorcaz', 'santos-de-la-humosa-los', 'serna-del-monte-la', 'serranillos-del-valle', 'sevilla-la-nueva', 'somosierra', 'soto-del-real', 'talamanca-de-jarama', 'tielmes', 'titulcia', 'torrejon-de-ardoz', 'torrejon-de-la-calzada', 'torrejon-de-velasco', 'torrelaguna', 'torrelodones', 'torremocha-de-jarama', 'torres-de-la-alameda', 'tres-cantos', 'valdaracete', 'valdeavero', 'valdelaguna', 'valdemanco', 'valdemaqueda', 'valdemorillo', 'valdemoro', 'valdeolmos-alalpardo', 'valdepielagos', 'valdetorres-de-jarama', 'valdilecha', 'valverde-de-alcala', 'velilla-de-san-antonio', 'vellon-el', 'venturada', 'villaconejos', 'villa-del-prado', 'villalbilla', 'villamanrique-de-tajo', 'villamanta', 'villamantilla', 'villanueva-de-la-canada', 'villanueva-del-pardillo', 'villanueva-de-perales', 'villar-del-olmo', 'villarejo-de-salvanes', 'villaviciosa-de-odon', 'villavieja-del-lozoya', 'zarzalejo']
municipios = []
geolocator = Nominatim(user_agent="lab-geolocation-GRO") #User agent debe ser lo que quieras
for i in tqdm(range(0,len(lista_municipios))):
    location = geolocator.geocode(lista_municipios[i])
    name = location.address.split(",")[0]
    municipios.append(name)

100%|██████████| 179/179 [01:30<00:00,  1.97it/s]


Hemos tardado 100.28039574623108 segundos
Hemos tardado 91.72400283813477 segundos

In [7]:
lista_municipios = ['acebeda-la']
municipios = []
geolocator = Nominatim(user_agent="lab-geolocation-GRO") #User agent debe ser lo que quieras
for i in tqdm(range(0,len(lista_municipios))):
    location = geolocator.geocode(lista_municipios[i])
    name = location.address.split(",")[0]
    municipios.append(name)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  4.41it/s]


In [4]:
municipios[-1] = municipios[-1].strip(" (estación)")

In [7]:
municipios = ['La Acebeda', 'Ajalvir', 'Alameda del Valle', 'El Álamo', 'Alcalá de Henares', 'Alcobendas', 'Alcorcón', 'Aldea del Fresno', 'Algete', 'Alpedrete', 'Ambite', 'Anchuelo', 'Aranjuez', 'Arganda del Rey', 'Arroyomolinos', 'El Atazar', 'Batres', 'Becerril de la Sierra', 'Belmonte de Tajo', 'El Berrueco', 'Berzosa del Lozoya', 'Boadilla del Monte', 'El Boalo', 'Braojos', 'Brea de Tajo', 'Brunete', 'Buitrago del Lozoya', 'Bustarviejo', 'Cabanillas de la Sierra', 'Cabrera Lane', 'Cadalso de los Vidrios', 'Camarma de Esteruelas', 'Campo Real', 'Canencia', 'Carabaña', 'Casarrubuelos', 'Cenicientos', 'Cercedilla', 'Cervera de Buitrago', 'Chapinería', 'Chinchón', 'Ciempozuelos', 'Cobeña', 'Collado Mediano', 'Collado Villalba', 'Colmenar del Arroyo', 'Colmenar de Oreja', 'Colmenarejo', 'Colmenar Viejo', 'Corpa', 'Coslada', 'Cubas de la Sagra', 'Daganzo de Arriba', 'Real Monasterio de San Lorenzo de El Escorial', 'Estremera', 'Fresnedillas de la Oliva', 'Fresno de Torote', 'Fuenlabrada', 'Fuente el Saz de Jarama', 'Fuentidueña de Tajo', 'Galapagar', 'Garganta de los Montes', 'Gargantilla del Lozoya y Pinilla de Buitrago', 'Gascones', 'Getafe', 'Griñón', 'Guadalix de la Sierra', 'Guadarrama', 'La Hiruela', 'Horcajo de la Sierra-Aoslos', 'Horcajuelo de la Sierra', 'Hoyo de Manzanares', 'Humanes de Madrid', 'Leganes', 'Loeches', 'Lozoya', 'Lozoyuela-Navas-Sieteiglesias', 'Madarcos', 'Madrid', 'Majadahonda', 'Manzanares el Real', 'Meco', 'Mejorada del Campo', 'Miraflores de la Sierra', 'El Molar', 'Molinos Los', 'Montejo de la Sierra', 'Moraleja de Enmedio', 'Moralzarzal', 'Morata de Tajuña', 'Móstoles', 'Navacerrada', 'Navalafuente', 'Navalagamella', 'Navalcarnero', 'Navarredonda y San Mamés', 'Navas del Rey', 'Nuevo Baztán', 'Olmeda de las Fuentes', 'Orusco de Tajuña', 'Paracuellos de Jarama', 'Parla', 'Patones', 'Pedrezuela', 'Pelayos de la Presa', 'Perales de Tajuña', 'Pezuela de las Torres', 'Pinilla del Valle', 'Pinto', 'Piñuécar-Gandullas', 'Pozuelo de Alarcón', 'Pozuelo del Rey', 'Prádena del Rincón', 'Puebla de la Sierra', 'Manjirón', 'Quijorna', 'Rascafría', 'Redueña', 'Ribatejada', 'Rivas-Vaciamadrid', 'Robledillo de la Jara', 'Robledo de Chavela', 'Robregordo', 'Las Rozas de Madrid', 'Rozas de Puerto Real', 'San Agustín del Guadalix', 'San Fernando de Henares', 'San Lorenzo de El Escorial', 'San Martín de la Vega', 'San Martín de Valdeiglesias', 'San Sebastián de los Reyes', 'Santa María de la Alameda', 'Santorcaz', 'Los Santos de la Humosa', 'La Serna del Monte', 'Serranillos del Valle', 'Sevilla la Nueva', 'Somosierra', 'Soto del Real', 'Talamanca de Jarama', 'Tielmes', 'Titulcia', 'Torrejón de Ardoz', 'Torrejón de la Calzada', 'Torrejón de Velasco', 'Torrelaguna', 'Torrelodones', 'Torremocha de Jarama', 'Torres de la Alameda', 'Tres Cantos', 'Valdaracete', 'Valdeavero', 'Valdelaguna', 'Valdemanco', 'Valdemaqueda', 'Valdemorillo', 'Valdemoro', 'Valdeolmos-Alalpardo', 'Valdepiélagos', 'Valdetorres de Jarama', 'Valdilecha', 'Valverde de Alcalá', 'Velilla de San Antonio', 'El Vellón', 'Venturada', 'Villaconejos', 'Villa del Prado', 'Villalbilla', 'Villamanrique de Tajo', 'Villamanta', 'Villamantilla', 'Villanueva de la Cañada', 'Villanueva del Pardillo', 'Villanueva de Perales', 'Villar del Olmo', 'Villarejo de Salvanés', 'Villaviciosa de Odón', 'Villavieja del Lozoya', 'Zarzalejo']
len(municipios)

179

In [5]:
municipios

['La Acebeda',
 'Ajalvir',
 'Alameda del Valle',
 'El Álamo',
 'Alcalá de Henares',
 'Alcobendas',
 'Alcorcón',
 'Aldea del Fresno',
 'Algete',
 'Alpedrete',
 'Ambite',
 'Anchuelo',
 'Aranjuez',
 'Arganda del Rey',
 'Arroyomolinos',
 'El Atazar',
 'Batres',
 'Becerril de la Sierra',
 'Belmonte de Tajo',
 'El Berrueco',
 'Berzosa del Lozoya',
 'Boadilla del Monte',
 'El Boalo',
 'Braojos',
 'Brea de Tajo',
 'Brunete',
 'Buitrago del Lozoya',
 'Bustarviejo',
 'Cabanillas de la Sierra',
 'Cabrera Lane',
 'Cadalso de los Vidrios',
 'Camarma de Esteruelas',
 'Campo Real',
 'Canencia',
 'Carabaña',
 'Casarrubuelos',
 'Cenicientos',
 'Cercedilla',
 'Cervera de Buitrago',
 'Chapinería',
 'Chinchón',
 'Ciempozuelos',
 'Cobeña',
 'Collado Mediano',
 'Collado Villalba',
 'Colmenar del Arroyo',
 'Colmenar de Oreja',
 'Colmenarejo',
 'Colmenar Viejo',
 'Corpa',
 'Coslada',
 'Cubas de la Sagra',
 'Daganzo de Arriba',
 'Real Monasterio de San Lorenzo de El Escorial',
 'Estremera',
 'Fresnedillas de l

In [62]:
municipios_seleccionados = random.sample(municipios,5)

In [ ]:
estaciones_meteorologicas = []
for i in range(0,len(municipios_seleccionados)):
    municipio = municipios[i]
    driver = webdriver.Chrome() #Para Abrir chrome solo ejecutar once
    url_wunder = f"https://www.wunderground.com/weather/es/{municipio}"
    driver.get(url_wunder)
    #Maximizar ventana
    driver.maximize_window() 
    print("intentamos seleccionar Cookies")
    select_cookies = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(select_cookies)
    try:
        aceptar_cookies = driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button")
        aceptar_cookies.click()
        print("Encontrado")
    except:
        print("AY MI CUQUI, no la encuentro")
        driver.quit()
        driver = webdriver.Chrome() #Para Abrir chrome solo ejecutar once
        driver.get(url_wunder)
            #Maximizar ventana
        driver.maximize_window() 
        print("intentamos seleccionar Cookies")
        select_cookies = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
        driver.switch_to.frame(select_cookies)
        try:
            aceptar_cookies = driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button")
            aceptar_cookies.click()
            print("Encontrado")
            driver.switch_to.default_content()
            sleep(5)
            driver.find_element("css selector","#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name").click()
            sleep(6)
            nombre_estacion = driver.find_element("css selector", f"#inner-content > div.region-content-top > app-dashboard-header > div.dashboard__header.small-12.ng-star-inserted > div > div.heading > h1").text.split(" - ")[1]
            estaciones_meteorologicas.append(nombre_estacion)
            driver.quit()

            df = pd.DataFrame({
                "Municipio": municipios_seleccionados,
                "Estación Meteorológica": estaciones_meteorologicas,
            })
            driver.quit()
        except:
            print("AY MI CUQUI, no la encuentro")
            driver.quit()


    driver.switch_to.default_content()
    sleep(5)
    driver.find_element("css selector","#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name").click()
    sleep(6)
    nombre_estacion = driver.find_element("css selector", f"#inner-content > div.region-content-top > app-dashboard-header > div.dashboard__header.small-12.ng-star-inserted > div > div.heading > h1").text.split(" - ")[1]
    estaciones_meteorologicas.append(nombre_estacion)
    driver.quit()

df = pd.DataFrame({
    "Municipio": municipios_seleccionados,
    "Estación Meteorológica": estaciones_meteorologicas,
})

df

# Listas del dataframe individuales

In [ ]:
municipios_seleccionados
estaciones_meteorologicas

In [67]:
df.to_csv("datos/municipios_to_work.csv")


# Toca sacar tabla de los datos mensuales

In [ ]:
df = pd.read_csv("datos/municipios_to_work.csv",index_col="Unnamed: 0")
df

In [27]:
municipios_seleccionados = df["Municipio"].to_list()
estaciones_meteorologicas = df["Estación Meteorológica"].to_list()


In [31]:
fechas = ["2024-1-1/2024-1-1","2024-2-1/2024-2-1","2024-3-1/2024-3-1","2024-4-1/2024-4-1","2024-5-1/2024-5-1","2024-6-1/2024-6-1","2024-7-1/2024-7-1","2024-8-1/2024-8-1","2024-9-1/2024-9-1","2024-10-1/2024-10-1"]

In [ ]:
df = pd.DataFrame()

for e in range(0,len(estaciones_meteorologicas)):
    estacion_meteorologica = estaciones_meteorologicas[e]
    df_temp = pd.DataFrame()
    for d in range(0,len(fechas)):
        driver = webdriver.Chrome() #Para Abrir chrome solo ejecutar once
        url_wunder = f"https://www.wunderground.com/dashboard/pws/{estacion_meteorologica}/table/{fechas[d]}/monthly"
        driver.get(url_wunder)
        #Maximizar ventana
        driver.maximize_window() 
        print("intentamos seleccionar Cookies")
        select_cookies = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
        driver.switch_to.frame(select_cookies)
        try:
            aceptar_cookies = driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button")
            aceptar_cookies.click()
            print("Encontrado")
        except:
            print("AY MI CUQUI, no la encuentro")

        driver.switch_to.default_content()
        sleep(5)

        tabla = driver.find_element("css selector","#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div > div")
        tabla_text = tabla.text
        tabla_split = tabla_text.split("\n")
        tabla_df = pd.DataFrame(tabla_split)
        df_temp = pd.concat([df_temp,tabla_df],axis=0)
        driver.quit()
        sleep(6)
    df_temp["Municipio"] = municipios_seleccionados[e]
    df_temp = df_temp.reindex(columns=["Municipio",0])
    df = pd.concat([df,df_temp],axis=0)
    


In [ ]:
backup = df

In [55]:
df = backup

In [68]:
df_split = df[0].str.split(expand=True)
df_split
df_munis = df
df_munis = df.drop(columns=[0])
df_concat = pd.concat([df_munis,df_split],axis=1)
df_concat

In [83]:
df = df_concat

In [ ]:
df = df.reset_index(drop=True)


In [ ]:
indices_fuera = [0, 32, 62, 94, 125, 157, 188, 220, 252, 283, 300, 332, 362, 394, 425, 457, 488, 520, 
552, 583, 600, 632, 662, 694, 725, 757, 788, 820, 852, 883, 900, 932, 962, 994, 
1025, 1057, 1088, 1120, 1152, 1183, 1200, 1232, 1262, 1294, 1325, 1357, 1388, 1420, 
1452, 1483]

df_selected = df.loc[indices_fuera]
df_selected = df
df_selected2 = df_selected.drop(index=indices_fuera)
df = df_selected2.reset_index(drop=True)


In [ ]:
df

In [129]:
df.to_csv("datos/tabla_a_limpiar.csv")

In [39]:
df = pd.read_csv("datos/tabla_a_limpiar.csv",index_col="Unnamed: 0")


In [40]:
df = df.drop(columns=["2","4","6","8","10","12","14","16","18","20","22","24","26","28","30"])
columnas = ["Date","Temperature High (Fº)", "Temperature Avg (Fº)","Temperature Low (Fº)","Dew Point High (Fº)",
            "Dew Point Avg (Fº)","Dew Point Low (Fº)","Humidity High (%)","Humidity Avg (%)","Humidity Low (%)",
            "Speed High (mph)","Speed Avg (mph)","Speed Low (mph)","Pressure High (in)","Pressure Low (in)","Precipitation Accumulation Sum (in)"]
df.rename(columns={"0":columnas[0],"1":columnas[1],"3":columnas[2],"5":columnas[3],
                   "7":columnas[4],"9":columnas[5],"11":columnas[6],"13":columnas[7],
                   "15":columnas[8],"17":columnas[9],"19":columnas[10],"21":columnas[11],
                   "23":columnas[12],"25":columnas[13],"27":columnas[14],"29":columnas[15]},inplace=True)
df["Date"] = pd.to_datetime(df["Date"],format="mixed")
df["Humidity High (%)"] = df["Humidity High (%)"].astype(int)
df["Humidity Avg (%)"] = df["Humidity Avg (%)"].astype(int)
df["Humidity Low (%)"] = df["Humidity Low (%)"].astype(int)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1450 entries, 0 to 1449
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Municipio                            1450 non-null   object        
 1   Date                                 1450 non-null   datetime64[ns]
 2   Temperature High (Fº)                1450 non-null   float64       
 3   Temperature Avg (Fº)                 1450 non-null   float64       
 4   Temperature Low (Fº)                 1450 non-null   float64       
 5   Dew Point High (Fº)                  1450 non-null   float64       
 6   Dew Point Avg (Fº)                   1450 non-null   float64       
 7   Dew Point Low (Fº)                   1450 non-null   float64       
 8   Humidity High (%)                    1450 non-null   int64         
 9   Humidity Avg (%)                     1450 non-null   int64         
 10  Humidity Low (%) 

In [43]:
df.head()

,Municipio,Date,Temperature High (Fº),Temperature Avg (Fº),Temperature Low (Fº),Dew Point High (Fº),Dew Point Avg (Fº),Dew Point Low (Fº),Humidity High (%),Humidity Avg (%),Humidity Low (%),Speed High (mph),Speed Avg (mph),Speed Low (mph),Pressure High (in),Pressure Low (in),Precipitation Accumulation Sum (in)
0,Santorcaz,2024-01-01,48.9,39.6,34.2,38.1,31.6,19.9,97,75,33,8.5,0.9,0.0,30.04,29.92,0.00
1,Santorcaz,2024-01-02,41.0,35.3,32.0,40.6,34.7,30.7,99,98,93,4.9,0.4,0.0,30.04,29.94,0.06
2,Santorcaz,2024-01-03,52.0,45.4,40.6,49.6,44.0,38.3,99,95,82,9.6,0.7,0.0,29.99,29.91,0.24
3,Santorcaz,2024-01-04,45.7,40.6,34.2,43.2,39.2,33.6,99,95,82,6.0,0.3,0.0,29.98,29.68,0.05
4,Santorcaz,2024-01-05,41.5,36.5,34.2,38.5,33.5,31.3,98,89,76,14.3,2.7,0.0,29.79,29.61,0.29


In [44]:
df.to_csv("datos/tabla_hecha.csv")